# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [ ]:
import pandas as pd

In [ ]:
# Carregando o dataset
df = pd.read_csv('previsao_de_renda.csv')

# Removendo as colunas utilidade
colunas_para_remover = ['data_ref', 'id_cliente', 'Unnamed: 0']
df = df.drop(columns=colunas_para_remover)

df

In [ ]:
# Contando os dados faltantes em cada coluna
print(df.isnull().sum())

In [ ]:
# Removendo as linhas com dados faltantes
df = df.dropna()

df

In [ ]:
df.info()

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [ ]:
from sklearn.model_selection import train_test_split

# Definindo as variáveis preditoras (X) e a variável alvo (y)
X = df.drop('renda', axis=1)
y = df['renda']

# Separando os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Verificando os tamanhos dos conjuntos
print(f"Tamanho do conjunto de treinamento: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import numpy as np

# Identificar colunas categóricas e numéricas
colunas_categoricas = X.select_dtypes(include=['object', 'category']).columns
colunas_numericas = X.select_dtypes(include=['int64', 'float64']).columns

# Criar um preprocessador
preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(drop='first'), colunas_categoricas)
    ])

# Criar um pipeline com o preprocessador e o modelo Ridge
pipeline = Pipeline([
    ('preprocessador', preprocessador),
    ('regressor', Ridge())
])

# Lista de valores alpha para testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Dicionário para armazenar os resultados
resultados = {}

# Testando cada valor de alpha
for alpha in alphas:
    # Configurando o alpha no modelo Ridge
    pipeline.set_params(regressor__alpha=alpha)
    
    # Treinando o modelo
    pipeline.fit(X_train, y_train)
    
    # Fazendo previsões no conjunto de teste
    y_pred = pipeline.predict(X_test)
    
    # Calculando o R² e armazenando o resultado
    r2 = r2_score(y_test, y_pred)
    resultados[alpha] = r2

# Imprimindo os resultados
for alpha, r2 in resultados.items():
    print(f"Alpha: {alpha}, R²: {r2:.4f}")

# Encontrando o melhor modelo
melhor_alpha = max(resultados, key=resultados.get)
print(f"\nMelhor modelo: Alpha = {melhor_alpha}, R² = {resultados[melhor_alpha]:.4f}")

In [ ]:
from sklearn.linear_model import Lasso

# Identificar colunas categóricas e numéricas
colunas_categoricas = X.select_dtypes(include=['object', 'category']).columns
colunas_numericas = X.select_dtypes(include=['int64', 'float64']).columns

# Criar um preprocessador
preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(drop='first'), colunas_categoricas)
    ])

# Criar um pipeline com o preprocessador e o modelo Lasso
pipeline = Pipeline([
    ('preprocessador', preprocessador),
    ('regressor', Lasso())
])

# Lista de valores alpha para testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Dicionário para armazenar os resultados
resultados = {}

# Testando cada valor de alpha
for alpha in alphas:
    # Configurando o alpha no modelo Lasso
    pipeline.set_params(regressor__alpha=alpha)
    
    # Treinando o modelo
    pipeline.fit(X_train, y_train)
    
    # Fazendo previsões no conjunto de teste
    y_pred = pipeline.predict(X_test)
    
    # Calculando o R² e armazenando o resultado
    r2 = r2_score(y_test, y_pred)
    resultados[alpha] = r2

# Imprimindo os resultados
for alpha, r2 in resultados.items():
    print(f"Alpha: {alpha}, R²: {r2:.4f}")

# Encontrando o melhor modelo
melhor_alpha = max(resultados, key=resultados.get)
print(f"\nMelhor modelo: Alpha = {melhor_alpha}, R² = {resultados[melhor_alpha]:.4f}")

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# Identificar colunas categóricas e numéricas
colunas_categoricas = X.select_dtypes(include=['object', 'category']).columns
colunas_numericas = X.select_dtypes(include=['int64', 'float64']).columns

# Criar um preprocessador
preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(drop='first'), colunas_categoricas)
    ])

# Criar um pipeline com o preprocessador, seletor de características e regressão linear
pipeline = Pipeline([
    ('preprocessador', preprocessador),
    ('selector', SequentialFeatureSelector(LinearRegression(), n_features_to_select='auto', direction='forward')),
    ('regressor', LinearRegression())
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = pipeline.predict(X_test)

# Calcular o R² no conjunto de teste
r2 = r2_score(y_test, y_pred)

print(f"R² do modelo stepwise no conjunto de teste: {r2:.4f}")

# Obter as características selecionadas
caracteristicas_selecionadas = pipeline.named_steps['selector'].get_support()
nomes_caracteristicas = pipeline.named_steps['preprocessador'].get_feature_names_out()
caracteristicas_finais = nomes_caracteristicas[caracteristicas_selecionadas]

print("\nCaracterísticas selecionadas:")
for caracteristica in caracteristicas_finais:
    print(caracteristica)

Eu diria que o modelo Ridge ou Lasso com alpha = 0.001 aparenta ser ligeiramente melhor, pois:
Eles oferecem uma pequena regularização, o que pode ajudar a prevenir overfitting.
Mantêm todas as variáveis no modelo, o que pode ser útil se todas as variáveis forem consideradas importantes do ponto de vista do negócio.
No entanto, se a interpretabilidade e a simplicidade do modelo forem prioridades, o modelo stepwise poderia ser uma escolha igualmente boa, já que ele seleciona automaticamente as características mais relevantes.
É importante notar que, para uma decisão final, seria útil considerar outros fatores como a estabilidade do modelo em diferentes subconjuntos de dados, a facilidade de implementação e manutenção, e as necessidades específicas do projeto ou negócio.

In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# Identificar colunas categóricas e numéricas
colunas_categoricas = X.select_dtypes(include=['object', 'category']).columns
colunas_numericas = X.select_dtypes(include=['int64', 'float64']).columns

# Criar um preprocessador
preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(drop='first'), colunas_categoricas)
    ])

# Criar um pipeline com o preprocessador e a árvore de decisão
pipeline = Pipeline([
    ('preprocessador', preprocessador),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = pipeline.predict(X_test)

# Calcular o R² no conjunto de teste
r2 = r2_score(y_test, y_pred)

print(f"R² da árvore de regressão no conjunto de teste: {r2:.4f}")

R² da árvore de regressão no conjunto de teste: 0.2258
